In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
from scipy.integrate import odeint

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

from smt.sampling_methods import LHS
from scipy.io import savemat

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())


cuda:0


In [2]:
tstart= 0 
tstop= 60 

increment = 0.1 
# Initial condition
x_init= [0,0] 

t = np.arange(tstart,tstop+1,increment)

c = 1 # Damping constant 
k = 5 # Stiffness of the spring
m = 5 # Mass 
F0 = 100

def mydiff(x, t):    
    F =F0
    
    dx1dt = x[1] 
    dx2dt = (F -c*x[1] -k*x[0])/m
    dxdt= [dx1dt, dx2dt] 
    
    return dxdt 

x_full_sol = odeint(mydiff, x_init, t) 

x_sol = x_full_sol[:,0]
v_sol = x_full_sol[:,1] 

In [3]:
level = "low"
label = "1D_SMD_stan_" + level

loss_thresh = 0.005

t_test = t.reshape(-1,1)
t_test_tensor = torch.from_numpy(t_test).float().to(device)

# bc1_t = t[0].reshape(-1,1)
# bc1_x = x_sol[0].reshape(-1,1)
# t_bc1_train = torch.from_numpy(bc1_t).float().to(device)
# x_bc1_train = torch.from_numpy(bc1_x).float().to(device)

bc1_t = t.reshape(-1,1)
bc1_x = x_sol.reshape(-1,1)
t_bc1_train = torch.from_numpy(bc1_t).float().to(device)
x_bc1_train = torch.from_numpy(bc1_x).float().to(device)


bc2_t = t[0].reshape(-1,1)
t_bc2_train = torch.from_numpy(bc2_t).float().to(device)
bc2_val = v_sol[0].reshape(-1,1)
bc2_val =torch.from_numpy(bc2_val).float().to(device)

t_test = t.reshape(-1,1)
t_test_tensor = torch.from_numpy(t_test).float().to(device)
x_true = x_sol
x_true_norm = np.linalg.norm(x_true,2)

# Domain bounds
lb = np.array(t[0]) 
ub = np.array(t[-1]) 

#torch.autograd.set_detect_anomaly(True)

In [4]:
def colloc_pts(N_f,seed):
    
    t01 = np.array([[0.0, 1.0]])
    sampling = LHS(xlimits=t01,random_state =seed)

    t_coll = lb + (ub-lb)*sampling(N_f)
    # t_coll = np.vstack((t_coll,)) # append training points to collocation points 

    return t_coll

In [5]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
    
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data) 
        
        self.beta = Parameter(0.0*torch.ones((50,len(layers)-2)))
        self.beta.requiresGrad = True
        
        self.k = Parameter(torch.tensor(1.0))
        self.k.requiresGrad = True
        self.c = Parameter(torch.tensor(1.0))
        self.c.requiresGrad = True
        self.m = Parameter(torch.tensor(1.0))
        self.m.requiresGrad = True
        
        self.lambdas = torch.ones((2,),device = device)
        self.lambda_alpha = 0.1
        
    'forward pass'
    def forward(self,t):
        if torch.is_tensor(t) != True:         
            t = torch.from_numpy(t)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        t = 2.0*(t - l_b)/(u_b - l_b) - 1.0 #feature scaling
        
        #convert to float
        a = t.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(z) + self.beta[:,i]*z*self.activation(z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC1(self,t,x):
                
        loss_bc1 = self.loss_function(self.forward(t), x)
                
        return loss_bc1
    
    def loss_BC2(self,t_bc2,bc2_val):
        g = t_bc2.clone()             
        g.requires_grad = True
        x = self.forward(g)    
            
        x_t = autograd.grad(x,g,torch.ones([t_bc2.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        dx_dt = x_t[:,[0]]
        
        bc2 = dx_dt
        
        loss_bc2= self.loss_function(bc2,bc2_val)

        return loss_bc2
    
    def loss_PDE(self,t_coll,f_hat):
             
        g = t_coll.clone()             
        g.requires_grad = True
  
        x = self.forward(g) 

        x_t = autograd.grad(x,g,torch.ones([t_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        x_tt = autograd.grad(x_t,g,torch.ones(t_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        dx_dt = x_t[:,[0]]
        
        dx2_d2t = x_tt[:,[0]]
        
        f = torch.abs(self.m)*dx2_d2t + torch.abs(self.c)*dx_dt + torch.abs(self.k)*x - F0
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    
    def loss(self,t_bc1,x_bc1,t_bc2,bc2_val,t_coll,f_hat):

        loss_bc1 = self.lambdas[0]*self.loss_BC1(t_bc1,x_bc1)
        loss_bc2 = self.lambdas[1]*self.loss_BC2(t_bc2,bc2_val)
        loss_f = self.loss_PDE(t_coll,f_hat)
        
        loss_val = loss_bc1 + loss_bc2 + loss_f
        
        return loss_val
    
    def lambda_update(self,t_bc1,x_bc1,t_bc2,bc2_val,t_coll,f_hat):
        loss_bc1 = self.lambdas[0]*self.loss_BC1(t_bc1,x_bc1)
        loss_bc1.backward()
        bc1_grads = []
        for param in self.parameters():
            bc1_grads.append(param.grad.view(-1))
        bc1_grads = torch.cat(bc1_grads)
        bc1_grads = torch.mean(torch.abs(bc1_grads))
        
        
        loss_bc2 = self.lambdas[1]*self.loss_BC2(t_bc2,bc2_val)
        loss_bc2.backward()
        bc2_grads = []
        for param in self.parameters():
            bc2_grads.append(param.grad.view(-1))
        bc2_grads = torch.cat(bc2_grads)
        bc2_grads = torch.mean(torch.abs(bc2_grads))
        
    
        loss_f = self.loss_PDE(t_coll,f_hat)
        loss_f.backward()
        f_grads = []
        for param in self.parameters():
            f_grads.append(param.grad.view(-1))   
        f_grads = torch.cat(f_grads)
        f_grads = torch.max(torch.abs(f_grads))
    
        self.lambdas[0] = (1.0-self.lambda_alpha)*self.lambdas[0] + self.lambda_alpha*f_grads/bc1_grads
        self.lambdas[1] = (1.0-self.lambda_alpha)*self.lambdas[1] + self.lambda_alpha*f_grads/bc2_grads
        
        return None
          
    'test neural network'
    
    def test(self):
        x_pred = self.forward(t_test_tensor)
        x_pred = x_pred.cpu().detach().numpy()

        return x_pred
    
    def test_loss(self):
        x_pred = self.test()
        
        test_mse = np.mean(np.square(x_pred.reshape(-1,1) - x_true.reshape(-1,1)))
        test_re = np.linalg.norm(x_pred.reshape(-1,1) - x_true.reshape(-1,1),2)/x_true_norm
        
        return test_mse, test_re

In [6]:
def train_step(t_bc1_train,x_bc1_train,t_bc2_train,bc2_val,t_coll,f_hat):
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(t_bc1_train,x_bc1_train,t_bc2_train,bc2_val,t_coll,f_hat)
        loss.backward()
        
        return loss

    optimizer.step(closure)
    PINN.lambda_update(t_bc1_train,x_bc1_train,t_bc2_train,bc2_val,t_coll,f_hat)

In [7]:
def data_update(loss_np):
    train_loss.append(loss_np)
    beta_val.append(PINN.beta.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [8]:
def train_model(max_iter,rep):
    print(rep) 
    torch.manual_seed(rep*123)
    start_time = time.time()
    thresh_flag = 0

    t_coll = colloc_pts(N_f,rep*11)
    t_coll =  torch.from_numpy(t_coll).float().to(device)
    
    f_hat = torch.zeros(t_coll.shape[0],1).to(device)
    for i in range(max_iter):      
        train_step(t_bc1_train,x_bc1_train,t_bc2_train,bc2_val,t_coll,f_hat)
         
        loss_np = PINN.loss(t_bc1_train,x_bc1_train,t_bc2_train,bc2_val,t_coll,f_hat).cpu().detach().numpy()
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test RE",test_re_loss[-1],"c",PINN.c.cpu().detach().numpy(),"k",PINN.k.cpu().detach().numpy(),"m",PINN.m.cpu().detach().numpy())
    
    elapsed_time[rep] = time.time() - start_time
    print('Training time: %.2f' % (elapsed_time[rep]))

In [9]:
max_reps = 1
max_iter = 500

train_loss_full = []
test_mse_full = []
test_re_full = []
beta_full = []
elapsed_time= np.zeros((max_reps,1))
time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))


for reps in range(max_reps):   
  train_loss = []
  test_mse_loss = []
  test_re_loss = []   
  beta_val = []

  torch.manual_seed(reps*36)
  N_f = 10000 #Total number of collocation points

  layers = np.array([1,50,50,50,1]) #9 hidden layers
  PINN = Sequentialmodel(layers)

  PINN.to(device)

  'Neural Network Summary'
  print(PINN)

  params = list(PINN.parameters())

  optimizer = torch.optim.LBFGS(PINN.parameters(), lr=1, 
                            max_iter = 10, 
                            max_eval = 15, 
                            tolerance_grad = 1e-6, 
                            tolerance_change = 1e-6, 
                            history_size = 100, 
                            line_search_fn = 'strong_wolfe')



  train_model(max_iter,reps)


  torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
  train_loss_full.append(train_loss)
  test_mse_full.append(test_mse_loss)
  test_re_full.append(test_re_loss)
  beta_full.append(beta_val)


  print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "beta": beta_full, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 2814.2231 Test RE 0.23160789671991563 c 0.90604484 k 4.4642177 m 0.999853
1 Train Loss 2332.3613 Test RE 0.20469604573569303 c 0.90235573 k 5.0369077 m 0.9998489
2 Train Loss 2529.4412 Test RE 0.2045745325117781 c 0.905337 k 5.035246 m 0.9998649
3 Train Loss 2561.0554 Test RE 0.20418174994099095 c 0.9183233 k 5.042436 m 0.999924
4 Train Loss 2369.828 Test RE 0.20355193720259496 c 0.92708325 k 4.9958124 m 0.9999584
5 Train Loss 2316.9355 Test RE 0.2010960175232826 c 0.9357603 k 5.025398 m 0.99987334
6 Train Loss 2003.2058 Test RE 0.19369151687716896 c 0.94791526 k 5.0886226 m 0.99952966
7 Train Loss 2061.764 Test RE 0.18364509946147212 c 0.

KeyboardInterrupt: 

In [ ]:
x_pred = PINN.test()
plt.plot(t,x_true,'b')
plt.plot(t,x_pred,'r--')